In [ ]:
# Included Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier,
                             GradientBoostingClassifier)
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve,
                             confusion_matrix)
from scipy import stats
from dataclasses import dataclass
from typing import Dict, Tuple, List

## Data Preprocessing and Cleanup

In [ ]:
# loading the dataset

df = pd.read_csv("data.csv")
df.head()


In [ ]:
print(df.info())

In [ ]:
# Filter for Tech Employees only
df = df[df['tech_company'] == "Yes"]

In [ ]:
df.shape

In [ ]:
# Keep only relevant columns

df = df[['workplace_resources','mh_employer_discussion','medical_coverage',
         'mh_share','gender','age']]

In [ ]:
# Drop missing values

df = df.dropna()

In [ ]:
# Data Preparation

df['high_comfort'] = np.where(df['mh_share'] >= 7, 1, 0)

In [ ]:
# Encode predictors

df['resources_binary'] = df['workplace_resources'].map({'Yes':1,'No':0})
# -1 for "I don't know"
df['resources_binary'] = df['resources_binary'].fillna(-1)

df['employer_binary'] = df['mh_employer_discussion'].map({'Yes':1,'No':0})
df['employer_binary'] = df['employer_binary'].fillna(-1)

df['coverage_binary']  = df['medical_coverage'].map({'Yes':1,'No':0})
df['coverage_binary']  = df['coverage_binary'].fillna(-1)

In [ ]:
## Engineered variable: Combined support

df['combined_support'] = ((df['resources_binary'] +
                           df['employer_binary'] +
                           df['coverage_binary']) >= 2).astype(int)

print("Dataset shape after cleaning:", df.shape)
print(df.head())

In [ ]:
# Encode gender as binary for modeling (Female=1, Male=0, Others=-1)
df['gender_binary'] = df['gender'].map({'Female':1, 'Male':0}).fillna(-1)

# Normalize age
df['age_scaled'] = (df['age'] - df['age'].mean()) / df['age'].std()

## Descriptive Analysis

In [ ]:
# Display the descriptive statistices of the cleaned data
df.describe()

In [ ]:
# Exploratory Data Analysis (EDA)
# Function to add percentages on bar plots
def plot_count_with_pct(x, hue, title):
    ax = sns.countplot(x=x, hue=hue, data=df, palette="Set2")
    total = len(df)
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{100*height/total:.1f}%',
                    (p.get_x() + p.get_width()/2, height),
                    ha='center', va='bottom', fontsize=9)
    plt.title(title)
    plt.xlabel(x.replace('_', ' ').replace('mh', 'Mental Health').title())
    plt.ylabel("Frequency")
    plt.legend(title="High Comfort", labels=["No (0)", "Yes (1)"])
    plt.show()

In [ ]:
plot_count_with_pct("workplace_resources","high_comfort","Comfort vs"
                    "Workplace Resources")

In [ ]:
plot_count_with_pct("mh_employer_discussion","high_comfort","Comfort vs"
                    "Employer Discussion")

In [ ]:
plot_count_with_pct("medical_coverage","high_comfort","Comfort vs"
                    "Medical Coverage")

In [ ]:
# Boxplot comfort levels
plt.figure(figsize=(6,4))
sns.boxplot(x="workplace_resources", y="mh_share", data=df,
            hue="workplace_resources", palette="pastel",legend=False)
plt.ylabel('Mental Health Share')
plt.xlabel("workplace_resources".replace('_', ' ')
                                .replace('mh', 'Mental Health').title())
plt.title("Comfort Scores by Workplace Resources")
plt.show()

In [ ]:
# Histogram distribution
plt.figure(figsize=(6,4))
sns.histplot(df['mh_share'], bins=10, kde=False, color="skyblue")
plt.title("Distribution of Comfort Sharing Scores")
plt.xlabel("Comfort Sharing Score (mh_share)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Stratified by gender
plt.figure(figsize=(6,4))
sns.barplot(x="gender", y="high_comfort", hue="workplace_resources",
            data=df, errorbar=None, palette="pastel", edgecolor="black")
plt.title("High Comfort by Gender & Resources")
plt.ylabel("% High Comfort")
plt.show()

In [ ]:
# Histogram of Age
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(df['age'], bins=20, color="skyblue", edgecolor='black')
plt.title("Distribution of Age")
plt.xlabel("Age")
plt.ylabel("Frequency")

plt.subplot(1,2,2)
df['age_group'] = pd.cut(df['age'], bins=[18,30,40,50,60,80],
                         labels=["18-29","30-39","40-49","50-59","60+"])
sns.barplot(x="age_group", y="mh_share", data=df,
            color="skyblue", errorbar=None)
plt.title("Average Comfort Score by Age Group")
plt.ylabel("Mean Comfort Score")
plt.show()

In [ ]:
# Average Comfort Score by Age Group


## Independent T-Test

In [ ]:
# Calculate Confidence interval for the Mean Difference
def get_confidence_interval(yes, no) -> tuple[float, float, float]:
    # Retrieve the mean of the two groups
    mean_diff = yes.mean() - no.mean()
    # Compute standard error
    se = np.sqrt(yes.var(ddof=1)/len(yes) + no.var(ddof=1)/len(no))
    # Calculate the 95% confidence interval
    ci_low, ci_high = mean_diff - 1.96*se, mean_diff + 1.96*se
    return mean_diff, ci_low, ci_high

In [ ]:
# Statistical Testing : Perform Welch's Independent T-Test
def welch_test(df, col1, col2, label):
    # Extract the data for the two comparing columns
    yes = df.loc[df[col1] == 1, col2].values
    no = df.loc[df[col1] == 0, col2].values
    # Perform Welch's T-Test
    t_stat, p_val = stats.ttest_ind(yes, no, equal_var=False)
    # Retrieve the significance test stats
    mean_diff, ci_low, ci_high = get_confidence_interval(yes, no)
    return {
            "Variable": label,
            "t(df)": f"{t_stat:.3f}",
            "p": p_val,
            "Mean Diff": f"{mean_diff:.2f}",
            "95% CI": f"[{ci_low:.2f}, {ci_high:.2f}]",
            "Significance": "Sig." if p_val < 0.05 else "Not Sig."
        }

In [ ]:
# Perfrom T-Test on all six predictors
rows = []
rows.append(welch_test(df, "resources_binary", "mh_share",
                      "Resources Binary"))
rows.append(welch_test(df, "employer_binary", "mh_share",
                       "Employer Discussion"))
rows.append(welch_test(df, "coverage_binary", "mh_share",
                       "Coverage Binary"))
table = pd.DataFrame(rows, columns=["Variable", "t(df)", "p", "Mean Diff",
                                    "95% CI", "Significance"])

# Report Test Results for all six predictors
display(table.style.set_caption("Table 3. Independent Samples t-Tests for"
                                "Mental Health Sharing (mh_share)"))

## Logistic Regression Model

In [ ]:
# Structures to store Test and Train Set Results
probs_test, preds_test = {}, {}
probs_train, preds_train = {}, {}

In [ ]:
# Print Model Information
def evaluate_model(y_test, y_pred, y_prob, model_name, total = 0):
    if total == 0:
        print(f"\n{model_name} Regression Performance:")
        print("Accuracy:", round(accuracy_score(y_test,y_pred),4))
        print("Precision:", round(precision_score(y_test,y_pred),4))
        print("Recall:", round(recall_score(y_test,y_pred),4))
        print("F1 Score:", round(f1_score(y_test,y_pred),4))
        print("AUC:", round(roc_auc_score(y_test,y_prob),4))

### Baseline Logistic Regression

In [ ]:
# Baseline Logistic Regression
model_name = "Baseline Logistic"

# Test-Train Split
# Split the data into training and testing sets (80% train, 20% test)
X1 = df[['resources_binary']].dropna()
y = df["high_comfort"].astype(int).copy()
X_train, X_test, y_train, y_test = train_test_split(
    X1, y, test_size=0.2, stratify=y, random_state=42
)

# Fit Logistic Regression Model on the training data
logit1 = LogisticRegression(max_iter=1000)
logit1.fit(X_train, y_train)

# Predict Train and Test Sets
# Predict binary class and probabilities labels for test set
pred_test1 = logit1.predict(X_test)
prob_test1 = logit1.predict_proba(X_test)[:,1]
pred_train1 = logit1.predict(X_train)
prob_train1 = logit1.predict_proba(X_train)[:,1]

# Store Train and Test Set Results
probs_train[model_name] = prob_train1
preds_train[model_name] = pred_train1
probs_test[model_name] = prob_test1
preds_test[model_name] = pred_test1

# Display Test Set Result
evaluate_model(y_test, pred_test1, prob_test1, model_name)

### Improved Logistic Regression

In [ ]:
# Improved Logistic Regression (with demographics)
model_name = "Improved Logistic"

# Test-Train Split for all 6 predictors
# Split the data into training and testing sets (80% train, 20% test)
X2 = df[['resources_binary','employer_binary','coverage_binary',
         'combined_support','gender_binary','age_scaled']]
X_train, X_test, y_train, y_test = train_test_split(
    X2,y,test_size=0.2,stratify=y,random_state=42
)

# Fit the Improved Logistic Regression Model on the training data
logit2 = LogisticRegression(max_iter=1000)
logit2.fit(X_train,y_train)

# Predict Train and Test Sets
# Predict binary class and probabilities labels for test set
pred_test2 = logit2.predict(X_test)
prob_test2 = logit2.predict_proba(X_test)[:,1]
pred_train2 = logit2.predict(X_train)
prob_train2 = logit2.predict_proba(X_train)[:,1]

# Store Train and Test Set Results
probs_train[model_name] = prob_train2
preds_train[model_name] = pred_train2
probs_test[model_name] = prob_test2
preds_test[model_name] = pred_test2

# Display Test Set Result
evaluate_model(y_test, pred_test2, prob_test2, model_name)

In [ ]:
def pred_summary(X: pd.DataFrame, y: pd.Series):
        Xc = sm.add_constant(X)
        logit = sm.Logit(y, Xc)
        result = logit.fit(disp=False)
        summ = result.summary2().tables[1].copy()
        summ = summ.rename(columns={"Coef.": "β", "Std.Err.": "SE", "P>|z|"
                                    : "p", "[0.025": "CI Low", "0.975]"
                                    :"CI High"})
        summ["OR"] = np.exp(summ["β"])
        # Order columns
        summ = summ[["β", "SE", "OR", "CI Low", "CI High", "p"]]
        return summ.round(3), result

In [ ]:
# Logistic Regression Coefficients
predictors = list(X_train.columns)
coefficients = pd.DataFrame({
    'Predictor': predictors,
    'Coefficient': logit2.coef_[0]
})
coefficients['Odds_Ratio'] = np.exp(coefficients['Coefficient'])
coefficients.sort_values(by='Odds_Ratio', ascending=False, inplace=True)
display(coefficients.style.set_caption("Table 4. Logistic Regression"
                                       "Coefficients and Odds Ratios "))

sm_table, sm_result = pred_summary(X_train, y_train)
display(sm_table.style.set_caption("Table 5. Predictor Logit"
                                   "Estimates (Train Set)"))

### Random Forest Model

In [ ]:
# Supporting Model: Random Forest
model_name = "Random Forest"

# Fit Random Forest Model
rf = RandomForestClassifier(n_estimators=200, class_weight="balanced",
                            random_state=42)
rf.fit(X_train,y_train)

# Predict Train and Test Sets
pred_test3 = rf.predict(X_test)
prob_test3 = rf.predict_proba(X_test)[:,1]
pred_train3 = rf.predict(X_train)
prob_train3 = rf.predict_proba(X_train)[:,1]

# Store Train and Test Set Results
probs_train[model_name] = prob_train3
preds_train[model_name] = pred_train3
probs_test[model_name] = prob_test3
preds_test[model_name] = pred_test3

# Display Test Set Result
evaluate_model(y_test, pred_test3, prob_test3, model_name)

### Gradient Boosting Model

In [ ]:
# Supporting Model: Gradient Boosting
model_name = "Gradient Boosting"

# Fit Gradient Boosting Model on the training data
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05,
                                random_state=42)
gb.fit(X_train,y_train)

# Predict Train and Test Sets
# Predict binary class and probabilities labels for test set
pred_test4 = gb.predict(X_test)
prob_test4 = gb.predict_proba(X_test)[:,1]
pred_train4 = gb.predict(X_train)
prob_train4 = gb.predict_proba(X_train)[:,1]

# Store Train and Test Set Results
probs_train[model_name] = prob_train4
preds_train[model_name] = pred_train4
probs_test[model_name] = prob_test4
preds_test[model_name] = pred_test4

# Display Test Set Result
evaluate_model(y_test, pred_test4, prob_test4, model_name)

### Model Analysis

In [ ]:
# Feature Importance
# Create a pandas series for the feature importances
# This gives each feature's contribution to the model
rf_importances = pd.Series(rf.feature_importances_, index=X_train.columns)
gb_importances = pd.Series(gb.feature_importances_, index=X_train.columns)

# Set up figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Random Forest plot
rf_importances.sort_values().plot(
    kind="barh", color="teal", ax=axes[0]
)
axes[0].set_title("Feature Importances – Random Forest", fontsize=12)
axes[0].set_xlabel("Importance")
axes[0].set_ylabel("Features")

# Gradient Boosting plot
gb_importances.sort_values().plot(
    kind="barh", color="orange", ax=axes[1]
)
axes[1].set_title("Feature Importances – Gradient Boosting", fontsize=12)
axes[1].set_xlabel("Importance")
axes[1].set_ylabel("")  # Hide duplicate y-labels

# Adjust layout and display importance plots
plt.tight_layout()
plt.show()

In [ ]:
# ROC Comparison
# Calculate the false positive rates, true postitive rate and the thresholds for each model
fpr1,tpr1,_ = roc_curve(y_test,prob_test1)
fpr2,tpr2,_ = roc_curve(y_test,prob_test2)
fpr_rf,tpr_rf,_ = roc_curve(y_test,prob_test3)
fpr_gb,tpr_gb,_ = roc_curve(y_test,prob_test4)

# Plot ROC Curves for each of the models
plt.plot(fpr1,tpr1,label=f"Baseline Logistic (AUC="
                      f"{roc_auc_score(y_test,prob_test1):.2f})",
                         color="blue")
plt.plot(fpr2,tpr2,label=f"Improved Logistic (AUC="
                      f"{roc_auc_score(y_test,prob_test2):.2f})",
                         color="green")
plt.plot(fpr_rf,tpr_rf,label=f"Random Forest (AUC="
                      f"{roc_auc_score(y_test,prob_test3):.2f})",
                         color="red")
plt.plot(fpr_gb,tpr_gb,label=f"Gradient Boosting (AUC="
                      f"{roc_auc_score(y_test,prob_test4):.2f})",
                         color="orange")

# Plot the lines for random guess and reference
plt.plot([0,1],[0,1],"k--")

# Set the axes lables and titles
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")

# Assign legend and display the plots
plt.legend()
plt.show()

In [ ]:
def plot_confusion_matrices(
        y_true: pd.Series, preds: Dict[str, np.ndarray]
    ):
        fig, axes = plt.subplots(2, 2, figsize=(14, 8))
        axes = axes.flatten()  # Flatten for easier iteration

        for ax, (model_name, y_pred) in zip(axes, preds.items()):
            cm = confusion_matrix(y_true, y_pred)
            acc = accuracy_score(y_true, y_pred)

            # Heatmap with gridlines between cells
            sns.heatmap(
                cm,
                annot=True,
                fmt="d",
                cmap="Blues",
                cbar=True,
                ax=ax,
                linewidths=1,
                linecolor="black",
                square=True,
                xticklabels=["Pred: Low","Pred: High"],
                yticklabels=["True: Low","True: High"]
            )

            ax.set_title(f"{model_name}\nAccuracy = {acc:.2f}",
                         fontsize=11, pad=8)
            ax.set_xlabel("Predicted Label")
            ax.set_ylabel("True Label")
            ax.tick_params(axis='x', labelrotation=45)
            ax.tick_params(axis='y', labelrotation=0)

        fig.add_artist(plt.Line2D((0.5, 0.5), (0, 1), color='black',
                                  linewidth=2,
                                  transform=fig.transFigure, zorder=10))
        fig.add_artist(plt.Line2D((0, 1), (0.5, 0.5), color='black',
                                  linewidth=2,
                                  transform=fig.transFigure, zorder=10))
        plt.subplots_adjust(wspace=0.4, hspace=0.4)
        plt.tight_layout()
        plt.show()

In [ ]:
plot_confusion_matrices(y_test, preds_test)

In [ ]:
# Train vs Test summaries
def performance_table(
         y_true: pd.Series, probs: Dict[str, np.ndarray],
         preds: Dict[str, np.ndarray]
    ) -> pd.DataFrame:
        rows = []
        # Loop through each model name and retrieve classification metrics
        for name in probs.keys():
            row = {
                "Model": name,
                "Accuracy": accuracy_score(y_true, preds[name]),
                "Precision": precision_score(y_true, preds[name],
                                             zero_division=0),
                "Recall": recall_score(y_true, preds[name], zero_division=0),
                "F1": f1_score(y_true, preds[name], zero_division=0),
                "AUC": roc_auc_score(y_true, probs[name]),
            }
            rows.append(row)
        df = pd.DataFrame(rows).sort_values("AUC", ascending=False)
        return (df[["Model", "Accuracy", "Precision", "Recall", "F1", "AUC"]]
               .round(3).reset_index(drop=True))

In [ ]:
# Model Performance Summary
# Train Set
train_table = performance_table(y_train, probs_train, preds_train)
# Test Set
test_table = performance_table(y_test, probs_test, preds_test)

# Display both train and test set tables
display(train_table.style.set_caption("Model Comparison Summary "
                                      "— Train Set").format(precision=3))
display(test_table.style.set_caption("Model Comparison Summary "
                                     "— Test Set").format(precision=3))


In [ ]:
# Display the heatmap for the train set
plt.figure(figsize=(8,4))
sns.heatmap(train_table.set_index("Model"), annot=True, fmt=".2f",
            cmap="Blues", cbar=False)
plt.title("Model Performance Heatmap for Train Set")
plt.show()

In [ ]:
# Display the heatmap for the test set
plt.figure(figsize=(8,4))
sns.heatmap(test_table.set_index("Model"), annot=True, fmt=".2f",
            cmap="Blues", cbar=False)
plt.title("Model Performance Heatmap for Test Set")
plt.show()

In [ ]:
# Cross-Validation

cv_scores = cross_val_score(LogisticRegression(max_iter=1000), X2, y, cv=5,
                            scoring="roc_auc")
print(f"\nCross-validated AUC for Improved Logistic Regression: "
      f"{cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

cv_scores_gb = cross_val_score(GradientBoostingClassifier(n_estimators=200,
                                                         learning_rate=0.05,
                                                         random_state=42),
                               X2, y, cv=5, scoring="roc_auc")
print(f"Cross-validated AUC for Gradient Boosting: {cv_scores_gb.mean():.3f} "
      f"± {cv_scores_gb.std():.3f}")